## Utils

In [3]:
# import torch
# from torch import cuda
# cuda.empty_cache()

In [4]:
import math
import math
import torch
import torch.nn as nn
import numpy as np
from skimage.metrics import peak_signal_noise_ratio
import os
import os.path
import numpy as np
import random
import h5py
import cv2
import glob
import torch.utils.data as udata
import torch.optim as optim
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter

In [5]:
def weights_init_kaiming(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.kaiming_normal(m.weight.data, a=0, mode="fan_in")
    elif classname.find("Linear") != -1:
        nn.init.kaiming_normal(m.weight.data, a=0, mode="fan_in")
    elif classname.find("BatchNorm") != -1:
        # nn.init.uniform(m.weight.data, 1.0, 0.02)
        m.weight.data.normal_(mean=0, std=math.sqrt(2.0 / 9.0 / 64.0)).clamp_(
            -0.025, 0.025
        )
        nn.init.constant(m.bias.data, 0.0)

In [6]:
def batch_PSNR(img,imclean,data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR =0 
    for i in range(Img.shape[0]):
        PSNR += peak_signal_noise_ratio(Iclean[i,:,:,:],Img[i,:,:,:],data_range=data_range)
    return (PSNR/Img.shape[0])


### Test PSNR

In [7]:
import torchvision
import cv2

nlm_path = "/home/kareem/hacking/research/AI_Love/denoising/CNN_TASK/SA&SK/_trad_methods_denoise/102NLM_0.95.bmp"

wtf_path = "/home/kareem/hacking/research/AI_Love/denoising/CNN_TASK/SA&SK/_trad_methods_denoise/102WFT_3.71.bmp"
noise_free = torchvision.io.read_image(
    "/home/kareem/hacking/research/AI_Love/denoising/CNN_TASK/SA&SK/Noise_free_image_0102.jpg"
)
noised_img = torchvision.io.read_image(
    "/home/kareem/hacking/research/AI_Love/denoising/CNN_TASK/SA&SK/Noisy_image_0102.jpg"
)
nlm_img = cv2.imread(nlm_path).transpose(2,0,1)
nlm_img = torchvision.transforms.Resize((550,550))(torch.tensor(nlm_img))
wtf_img =cv2.imread(wtf_path).transpose(2,0,1)
wtf_img = torchvision.transforms.Resize((550,550))(torch.tensor(wtf_img))
print(nlm_img.shape)

data_range = 255
noise_free = noise_free.unsqueeze(0)
noised_img = noised_img.unsqueeze(0)
nlm_img = nlm_img.unsqueeze(0)
wtf_img = wtf_img.unsqueeze(0)
noise_free.shape, noised_img.shape, nlm_img.shape, wtf_img.shape

torch.Size([3, 550, 550])


/home/kareem/mambaforge/envs/dinov2/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


(torch.Size([1, 3, 550, 550]),
 torch.Size([1, 3, 550, 550]),
 torch.Size([1, 3, 550, 550]),
 torch.Size([1, 3, 550, 550]))

In [8]:

# Compute the average PSNR
average_psnr = batch_PSNR(noised_img, noise_free, data_range)
average_nlm = batch_PSNR(nlm_img, noise_free, data_range)
average_wtf = batch_PSNR(wtf_img, noise_free, data_range)

print(f"Average PSNR: {average_psnr:.4f}")
print(f"Average PSNR NLM: {average_nlm:.4f}")
print(f"Average PSNR WTF: {average_wtf:.4f}")

Average PSNR: 20.4611
Average PSNR NLM: 6.5158
Average PSNR WTF: 6.8052


In [9]:
def data_augmentation(image, mode):
    out = np.transpose(image, (1,2,0))
    if mode == 0:
        # original
        out = out
    elif mode == 1:
        # flip up and down
        out = np.flipud(out)
    elif mode == 2:
        # rotate counterwise 90 degree
        out = np.rot90(out)
    elif mode == 3:
        # rotate 90 degree and flip up and down
        out = np.rot90(out)
        out = np.flipud(out)
    elif mode == 4:
        # rotate 180 degree
        out = np.rot90(out, k=2)
    elif mode == 5:
        # rotate 180 degree and flip
        out = np.rot90(out, k=2)
        out = np.flipud(out)
    elif mode == 6:
        # rotate 270 degree
        out = np.rot90(out, k=3)
    elif mode == 7:
        # rotate 270 degree and flip
        out = np.rot90(out, k=3)
        out = np.flipud(out)
    return np.transpose(out, (2,0,1))


## Models

In [10]:
import torch
import torch.nn as nn

In [11]:
class DnCNN(nn.Module):
    def __init__(self, channels, num_of_layers=17):
        super(DnCNN, self).__init__()
        kernel_size = 3
        padding = 1
        features = 64
        layers = []
        layers.append(
            nn.Conv2d(
                in_channels=channels,
                out_channels=features,
                kernel_size=kernel_size,
                padding=padding,
                bias=False,
            )
        )
        layers.append(nn.ReLU(inplace=True))
        for _ in range(num_of_layers - 2):
            layers.append(
                nn.Conv2d(
                    in_channels=features,
                    out_channels=features,
                    kernel_size=kernel_size,
                    padding=padding,
                    bias=False,
                )
            )
            layers.append(nn.BatchNorm2d(features))
            layers.append(nn.ReLU(inplace=True))
        layers.append(
            nn.Conv2d(
                in_channels=features,
                out_channels=channels,
                kernel_size=kernel_size,
                padding=padding,
                bias=False,
            )
        )
        self.dncnn = nn.Sequential(*layers)

    def forward(self, x):
        out = self.dncnn(x)
        return out

## Dataset

In [12]:
def normalize(data):
    return data/255.0

def Im2Patch(img,win,stride=1):
    k = 0
    endc = img.shape[0]
    endw = img.shape[1]
    endh = img.shape[2]
    patch = img[:,0:endw-win+0+1:stride,0:endh-win+0+1:stride]
    TotalPatNum = patch.shape[1] * patch.shape[2]
    Y = np.zeros([endc,win*win,TotalPatNum],np.float32)
    for i in range(win):
        for j in range(win):
            patch = img[:,i:endw-win+i+1:stride,j:endh-win+j+1:stride]
            Y[:,k,:] = np.array(patch[:]).reshape(endc,TotalPatNum)
            k = k + 1
    return Y.reshape([endc,win,win,TotalPatNum])


def prepare_data(data_path, patch_size, stride, aug_times=1):
    # train
    print("process training data")
    scales = [1, 0.9, 0.8, 0.7]
    #! Fix this to load .emb, jpeg, jpg images
    files = glob.glob(os.path.join(data_path, "train", "*.png"))
    files.sort()
    h5f = h5py.File("train.h5", "w")
    train_num = 0
    for i in range(len(files)):
        img = cv2.imread(files[i])
        h, w, c = img.shape
        for k in range(len(scales)):
            Img = cv2.resize(
                img,
                (int(h * scales[k]), int(w * scales[k])),
                interpolation=cv2.INTER_CUBIC,
            )
            Img = np.expand_dims(Img[:, :, 0].copy(), 0)
            Img = np.float32(normalize(Img))
            patches = Im2Patch(Img, win=patch_size, stride=stride)
            print(
                "file: %s scale %.1f # samples: %d"
                % (files[i], scales[k], patches.shape[3] * aug_times)
            )
            for n in range(patches.shape[3]):
                data = patches[:, :, :, n].copy()
                h5f.create_dataset(str(train_num), data=data)
                train_num += 1
                for m in range(aug_times - 1):
                    data_aug = data_augmentation(data, np.random.randint(1, 8))
                    h5f.create_dataset(
                        str(train_num) + "_aug_%d" % (m + 1), data=data_aug
                    )
                    train_num += 1
    h5f.close()
    print("\nprocess validation data")
    #! Validation data Fix this 
    files.clear()
    files = glob.glob(os.path.join(data_path, "Set12", "*.png"))
    files.sort()
    h5f = h5py.File("val.h5", "w")
    val_num = 0
    for i in range(len(files)):
        print("file: %s" % files[i])
        img = cv2.imread(files[i])
        img = np.expand_dims(img[:, :, 0], 0)
        img = np.float32(normalize(img))
        h5f.create_dataset(str(val_num), data=img)
        val_num += 1
    h5f.close()
    print("training set, # samples %d\n" % train_num)
    print("val set, # samples %d\n" % val_num)

In [13]:
prepare_data("/home/kareem/hacking/research/AI_Love/denoising/DcNN/data",patch_size=40,stride=10,aug_times=1)

process training data
file: /home/kareem/hacking/research/AI_Love/denoising/DcNN/data/train/test_001.png scale 1.0 # samples: 225
file: /home/kareem/hacking/research/AI_Love/denoising/DcNN/data/train/test_001.png scale 0.9 # samples: 169
file: /home/kareem/hacking/research/AI_Love/denoising/DcNN/data/train/test_001.png scale 0.8 # samples: 121
file: /home/kareem/hacking/research/AI_Love/denoising/DcNN/data/train/test_001.png scale 0.7 # samples: 81
file: /home/kareem/hacking/research/AI_Love/denoising/DcNN/data/train/test_002.png scale 1.0 # samples: 225
file: /home/kareem/hacking/research/AI_Love/denoising/DcNN/data/train/test_002.png scale 0.9 # samples: 169
file: /home/kareem/hacking/research/AI_Love/denoising/DcNN/data/train/test_002.png scale 0.8 # samples: 121
file: /home/kareem/hacking/research/AI_Love/denoising/DcNN/data/train/test_002.png scale 0.7 # samples: 81
file: /home/kareem/hacking/research/AI_Love/denoising/DcNN/data/train/test_003.png scale 1.0 # samples: 225
file: /h

In [14]:
class Dataset(udata.Dataset):
    def __init__(self, train=True):
        super(Dataset, self).__init__()
        self.train = train
        if self.train:
            h5f = h5py.File("train.h5", "r")
        else:
            h5f = h5py.File("val.h5", "r")
        self.keys = list(h5f.keys())
        random.shuffle(self.keys)
        h5f.close()

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, index):
        if self.train:
            h5f = h5py.File("train.h5", "r")
        else:
            h5f = h5py.File("val.h5", "r")
        key = self.keys[index]
        data = np.array(h5f[key])
        h5f.close()
        return torch.Tensor(data)

In [15]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
outf= "./logs"
mode = "S"
noiseL = 50
val_noiseL = 25
batchSize = 4
num_of_layers = 17
lr = 1e-3
epochs=10
milestone = 30

In [16]:
# Load dataset
print("Loading dataset ...\n")
dataset_train = Dataset(train=True)
dataset_val = Dataset(train=False)
loader_train = DataLoader(
    dataset=dataset_train, num_workers=4, batch_size=batchSize, shuffle=True
)
print("# of training samples: %d\n" % int(len(dataset_train)))
# Build model
net = DnCNN(channels=1, num_of_layers=num_of_layers)
net.apply(weights_init_kaiming)
criterion = nn.MSELoss(size_average=False)
# Move to GPU
device_ids = [0]
model = nn.DataParallel(net, device_ids=device_ids).cuda()
criterion.cuda()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# training
writer = SummaryWriter(outf)
step = 0
noiseL_B = [0, 55]  # ingnored when opt.mode=='S'
for epoch in range(epochs):
    if epoch < milestone:
        current_lr = lr
    else:
        current_lr = lr / 10.0
    # set learning rate
    for param_group in optimizer.param_groups:
        param_group["lr"] = current_lr
    print("learning rate %f" % current_lr)
    # train
    for i, data in enumerate(loader_train, 0):
        # training step
        model.train()
        model.zero_grad()
        optimizer.zero_grad()
        img_train = data
        if mode == "S":
            noise = torch.FloatTensor(img_train.size()).normal_(
                mean=0, std=noiseL / 255.0
            )
        if mode == "B":
            noise = torch.zeros(img_train.size())
            stdN = np.random.uniform(noiseL_B[0], noiseL_B[1], size=noise.size()[0])
            for n in range(noise.size()[0]):
                sizeN = noise[0, :, :, :].size()
                noise[n, :, :, :] = torch.FloatTensor(sizeN).normal_(
                    mean=0, std=stdN[n] / 255.0
                )
        imgn_train = img_train + noise
        img_train, imgn_train = Variable(img_train.cuda()), Variable(imgn_train.cuda())
        noise = Variable(noise.cuda())
        out_train = model(imgn_train)
        loss = criterion(out_train, noise) / (imgn_train.size()[0] * 2)
        loss.backward()
        optimizer.step()
        # results
        model.eval()
        out_train = torch.clamp(imgn_train - model(imgn_train), 0.0, 1.0)
        psnr_train = batch_PSNR(out_train, img_train, 1.0)
        print(
            "[epoch %d][%d/%d] loss: %.4f PSNR_train: %.4f"
            % (epoch + 1, i + 1, len(loader_train), loss.item(), psnr_train)
        )
        # if you are using older version of PyTorch, you may need to change loss.item() to loss.data[0]
        if step % 10 == 0:
            # Log the scalar values
            writer.add_scalar("loss", loss.item(), step)
            writer.add_scalar("PSNR on training data", psnr_train, step)
        step += 1
    ## the end of each epocha
    model.eval()
    # validate
    psnr_val = 0
    for k in range(len(dataset_val)):
        img_val = torch.unsqueeze(dataset_val[k], 0)
        noise = torch.FloatTensor(img_val.size()).normal_(
            mean=0, std=val_noiseL / 255.0
        )
        imgn_val = img_val + noise
        img_val, imgn_val = Variable(img_val.cuda(), volatile=True), Variable(
            imgn_val.cuda(), volatile=True
        )
        out_val = torch.clamp(imgn_val - model(imgn_val), 0.0, 1.0)
        psnr_val += batch_PSNR(out_val, img_val, 1.0)
    psnr_val /= len(dataset_val)
    print("\n[epoch %d] PSNR_val: %.4f" % (epoch + 1, psnr_val))
    writer.add_scalar("PSNR on validation data", psnr_val, epoch)
    # log the images
    out_train = torch.clamp(imgn_train - model(imgn_train), 0.0, 1.0)
    Img = utils.make_grid(img_train.data, nrow=8, normalize=True, scale_each=True)
    Imgn = utils.make_grid(imgn_train.data, nrow=8, normalize=True, scale_each=True)
    Irecon = utils.make_grid(out_train.data, nrow=8, normalize=True, scale_each=True)
    writer.add_image("clean image", Img, epoch)
    writer.add_image("noisy image", Imgn, epoch)
    writer.add_image("reconstructed image", Irecon, epoch)
    # save model
    torch.save(model.state_dict(), os.path.join(outf, "net.pth"))

Loading dataset ...

# of training samples: 154960



/tmp/ipykernel_260209/3161319601.py:4: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data, a=0, mode="fan_in")
/tmp/ipykernel_260209/3161319601.py:12: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.bias.data, 0.0)
/home/kareem/mambaforge/envs/dinov2/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


learning rate 0.001000
[epoch 1][1/38740] loss: 31.6343 PSNR_train: 14.4272
[epoch 1][2/38740] loss: 31.2700 PSNR_train: 14.6965
[epoch 1][3/38740] loss: 30.3939 PSNR_train: 14.8280
[epoch 1][4/38740] loss: 28.9016 PSNR_train: 15.4976
[epoch 1][5/38740] loss: 30.2285 PSNR_train: 14.8023
[epoch 1][6/38740] loss: 28.2910 PSNR_train: 14.8148
[epoch 1][7/38740] loss: 28.8015 PSNR_train: 14.9603
[epoch 1][8/38740] loss: 26.9579 PSNR_train: 14.8904
[epoch 1][9/38740] loss: 25.4840 PSNR_train: 14.9706
[epoch 1][10/38740] loss: 23.1313 PSNR_train: 14.4782
[epoch 1][11/38740] loss: 21.9095 PSNR_train: 15.0718
[epoch 1][12/38740] loss: 20.7826 PSNR_train: 14.7143
[epoch 1][13/38740] loss: 19.6188 PSNR_train: 15.3873
[epoch 1][14/38740] loss: 16.5695 PSNR_train: 14.9634
[epoch 1][15/38740] loss: 15.8624 PSNR_train: 14.9375
[epoch 1][16/38740] loss: 14.5102 PSNR_train: 14.5625
[epoch 1][17/38740] loss: 12.9315 PSNR_train: 14.6839
[epoch 1][18/38740] loss: 12.0267 PSNR_train: 14.4914
[epoch 1][19/3

KeyboardInterrupt: 

In [43]:
# "-test_data", type=str, default="Set12", help="test on Set12 or Set68"
# Build model
logdir = "/home/kareem/hacking/research/AI_Love/denoising/DcNN/logs"
print("Loading model ...\n")
net = DnCNN(channels=1, num_of_layers=num_of_layers)
device_ids = [0]
model = nn.DataParallel(net, device_ids=device_ids).cuda()
model.load_state_dict(torch.load(os.path.join(logdir, "net.pth")))
model.eval()
# load data info
print("Loading data info ...\n")
files_source = glob.glob(os.path.join("data", "Set12", "*.png"))
files_source.sort()
# process data
psnr_test = 0
for f in files_source:
    # image
    Img = cv2.imread(f)
    Img = normalize(np.float32(Img[:, :, 0]))
    Img = np.expand_dims(Img, 0)
    Img = np.expand_dims(Img, 1)
    ISource = torch.Tensor(Img)
    # noise
    noise = torch.FloatTensor(ISource.size()).normal_(
        mean=0, std=25.0/ 255.0
    )
    # noisy image
    INoisy = ISource + noise
    ISource, INoisy = Variable(ISource.cuda()), Variable(INoisy.cuda())
    with torch.no_grad():  # this can save much memory
        Out = torch.clamp(INoisy - model(INoisy), 0.0, 1.0)
    ## if you are using older version of PyTorch, torch.no_grad() may not be supported
    # ISource, INoisy = Variable(ISource.cuda(),volatile=True), Variable(INoisy.cuda(),volatile=True)
    # Out = torch.clamp(INoisy-model(INoisy), 0., 1.)
    psnr = batch_PSNR(Out, ISource, 1.0)
    psnr_test += psnr
    print("%s PSNR %f" % (f, psnr))
psnr_test /= len(files_source)
print("\nPSNR on test data %f" % psnr_test)

Loading model ...

Loading data info ...

data/Set12/01.png PSNR 26.318962
data/Set12/02.png PSNR 29.528109
data/Set12/03.png PSNR 27.182668
data/Set12/04.png PSNR 25.554388
data/Set12/05.png PSNR 26.563950
data/Set12/06.png PSNR 25.260837
data/Set12/07.png PSNR 26.267699
data/Set12/08.png PSNR 28.783051
data/Set12/09.png PSNR 23.905912
data/Set12/10.png PSNR 26.322535
data/Set12/11.png PSNR 26.467454
data/Set12/12.png PSNR 25.742276

PSNR on test data 26.491487
